<a href="https://colab.research.google.com/github/fernandoGitHub/ML_Projects/blob/main/UNDP_Demographics_Data/UNDP_Demographics_Data_Analysis-Part_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **UNDP_Demographics_Data-Analysis-Part_01**

This notebook will exercise working with several csv files while focusing on cleaning, cleansing and complete missing values using dataframe functionality only

# Setup

In [1]:
!wget https://raw.githubusercontent.com/fernandoGitHub/MLOPS_GSD/main/MLOP_setup.py
import MLOP_setup

MLOP_setup.install_package('WGET')

--2022-05-22 14:00:13--  https://raw.githubusercontent.com/fernandoGitHub/MLOPS_GSD/main/MLOP_setup.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2176 (2.1K) [text/plain]
Saving to: ‘MLOP_setup.py’

MLOP_setup.py       100%[===================>]   2.12K  --.-KB/s    in 0s      

2022-05-22 14:00:13 (28.8 MB/s) - ‘MLOP_setup.py’ saved [2176/2176]

Installing wget ...
Package wget has been successfully installed
Reloading Packages


In [2]:
import os
import shutil
import wget

import numpy as np
import pandas as pd

import pprint
pp = pprint.PrettyPrinter()

In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Data Preparation

In [4]:
_RAW_DATA_DIR = './raw_data/'
_PROCESSED_DATA_DIR = './processed_data'
_ZIP_FILENAME = 'UNDP_Demographics_Data.zip'
_UTILS_FILENAME = 'UNDP_utils.py'

_DATA_REP_URL = 'https://github.com/fernandoGitHub/ML_Projects/raw/main/UNDP_Demographics_Data/data/'
_UTILS_REP_URL = 'https://github.com/fernandoGitHub/ML_Projects/raw/main/UNDP_Demographics_Data/'

_ZIP_FULL_PATH = os.path.join(_RAW_DATA_DIR, _ZIP_FILENAME)
_ZIP_URL_PATH = os.path.join(_DATA_REP_URL, _ZIP_FILENAME)

_UTILS_URL_PATH = os.path.join(_UTILS_REP_URL, _UTILS_FILENAME)
_UTILS_FULL_PATH = os.path.join('.', _UTILS_FILENAME)

if os.path.isdir('./sample_data'):
  shutil.rmtree('./sample_data')

if not os.path.isdir(_RAW_DATA_DIR):
  os.makedirs(_RAW_DATA_DIR)

if not os.path.isdir(_PROCESSED_DATA_DIR):
  os.makedirs(_PROCESSED_DATA_DIR)

if not os.path.isfile(_UTILS_FULL_PATH):
  file_name = wget.download(_UTILS_URL_PATH)
  print(f"Fetching from GitHub: {file_name} ...")

if not os.path.isfile(_ZIP_FULL_PATH):
  file_name = wget.download(_ZIP_URL_PATH)
  print(f"Fetching from GitHub: {file_name} ...")
  os.replace(os.path.join('.', _ZIP_FILENAME), _ZIP_FULL_PATH)

Fetching from GitHub: UNDP_utils.py ...
Fetching from GitHub: UNDP_Demographics_Data.zip ...


In [5]:
import UNDP_utils

In [6]:
from zipfile import ZipFile

with ZipFile(_ZIP_FULL_PATH, 'r') as zip:
  # printing all the contents of the zip file
  zip.printdir()

  # extracting all the files
  zip.extractall(_RAW_DATA_DIR)

File Name                                             Modified             Size
Median_age.csv                                 2022-05-13 15:11:54         9288
Old_age_dependency_ratio.csv                   2022-05-13 15:11:54        14159
Population _ages_65 _and _older.csv            2022-05-13 15:11:54        12651
Population_ages_15_64.csv                      2022-05-13 15:11:54        14059
Population_under_age_5.csv                     2022-05-13 15:11:54        12955
Sex_ratio_at_birth.csv                         2022-05-13 15:11:54         9570
Total_Population.csv                           2022-05-13 15:11:54        15523
Urban_Population.csv                           2022-05-13 15:11:54        16114
Young_age_dependency_ratio.csv                 2022-05-13 15:11:54        15756


# Dataset Preparation

First, let's review the columnns at each dataset

In [7]:
csv_file_list, csv_full_path_list = UNDP_utils.generate_file_list_from_dir(path=_RAW_DATA_DIR, filter = '.csv', display=False)

for file in csv_full_path_list:
  temp_df = pd.read_csv(file)
  pp.pprint (f"csv file: {file} - columns: {temp_df.columns}")

("csv file: ./raw_data/Urban_Population.csv - columns: Index(['HDI Rank', "
 "'Country', '1990', '1995', '2000', '2005', '2010', '2011',\n"
 "       '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019'],\n"
 "      dtype='object')")
("csv file: ./raw_data/Sex_ratio_at_birth.csv - columns: Index(['HDI Rank', "
 "'Country', '1990', '1995', '2000', '2005', '2010', '2015',\n"
 "       '2019'],\n"
 "      dtype='object')")
("csv file: ./raw_data/Total_Population.csv - columns: Index(['HDI Rank', "
 "'Country', '1990', '1995', '2000', '2005', '2010', '2011',\n"
 "       '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', "
 "'2030'],\n"
 "      dtype='object')")
("csv file: ./raw_data/Young_age_dependency_ratio.csv - columns: Index(['HDI "
 "Rank', 'Country', '1990', '1995', '2000', '2005', '2010', '2011',\n"
 "       '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019'],\n"
 "      dtype='object')")
("csv file: ./raw_data/Median_age.csv - columns: Index(['HDI

**Finding:** We can see some irregularities:
   1.- Median and Sex Ratio have less columns (every 5 years)
   2.- Total population has an estimation for 2030

**Next Tasks**
1.   Remove sex-ratio_at_birth from directory (not clear how it helps)
2.   Remove spaces from all CSV files, replace unknown by -1.0 and save a new copy
3.   Interpolate values for median dataframe and save to file
4.   Drop the 2030 column from total population as store it as a label dataframe

In [18]:
# Let's create some util functions

def generate_file_list_from_dir(path, filter = '*.*', display=False):
  """generate_list_from_dir(path, filter = '*.*', display=False) returns two lists of strings.
  The first includes the names of the csv files, the second the full paths. The filter parameter
  can be used to return only a given type of files.
  By default, the function doesn't filter any file"""

  file_list = os.listdir(path=path)

  if filter != '*.*':
    file_list = [file for file in file_list if filter in file]

  full_path_list = [os.path.join(path, file) for file in file_list]

  if display:
    print(file_list)

  return file_list, full_path_list


# Helper Function to remove spaces and replace '..' with -1
def clean_data(df):
  # We need to remove spaces from the dataframe prior to interpolation
  # And also to replace the unknown values (expressed as two consecutive points)
  # to an arbitrary value of -1
  # Finally we will convert the values to float
  for col in df.columns:
    df[col] = df[col].astype(str).str.strip().replace('..','-1')

  return df


# Helper function to set the columns type
def set_column_type(df, type, inclusive=None, exclusive=None):

  cols = df.columns
  if inclusive == None:
    cols = [col for col in cols if not(col in exclusive)]
  else:
    cols = [col for col in cols if (col in inclusive)]

  for col in cols:
    df[col] = df[col].astype(float)

  return df


# Helper function to replace unknown values by mean values for that year
def replace_unknown_by_mean(df):
  for col in df.columns:
    if df[col].dtype == 'float64':
      mean_value = np.round(df[df[col] != -1][col].mean(), 1)
      df[col] = df[col].replace(-1.0, mean_value)
  
  return df


# Helper function to create interpolated columns
def interpolate_columns(df, new_cols):
  
  def interpolate (y1, y2, x1, x2, x):
    return np.round(y1 + (y2-y1)/(x2-x1) * (x-x1), 1)

  for col in new_cols:
    if not (col in df.columns):
      # Adding a new column and creating the interpolation
      year = int(col)
      prev_year = year - year % 5
      next_year = prev_year + 5

      df[col] = np.vectorize(interpolate)(df[str(prev_year)], df[str(next_year)], prev_year, next_year, year)

  return df


def remove_empty_records(df, key):
  for col in df.columns:
    df = df[df[col] != key]

  return df

def remove_out_of_range_percentage(df):
  for col in df.columns:
    df = df[df[col] <= 100.0]
    df = df[df[col] >= 0.0]

  return df

In [9]:
# 1. Removing sex-ratio-at-birth
os.remove('./raw_data/Sex_ratio_at_birth.csv')

csv_file_list, csv_full_path_list = UNDP_utils.generate_file_list_from_dir(path=_RAW_DATA_DIR, filter = '.csv', display=True)

['Urban_Population.csv', 'Total_Population.csv', 'Young_age_dependency_ratio.csv', 'Median_age.csv', 'Population_ages_15_64.csv', 'Old_age_dependency_ratio.csv', 'Population_under_age_5.csv', 'Population _ages_65 _and _older.csv']


In [19]:
# 2. Remove spaces from all CSV files, replace unknown by column mean values and save a new copy
for file in csv_file_list:
  df = pd.read_csv(os.path.join(_RAW_DATA_DIR, file))
  df = df.drop('HDI Rank', axis = 1)
  df = df.set_index('Country')
  df = clean_data(df)
  df = set_column_type(df, type='float',exclusive=['Country'])
  df = remove_empty_records(df=df, key= -1)
  if file in ['Young_age_dependency_ratio.csv', 'Old_age_dependency_ratio.csv']:
    df = remove_out_of_range_percentage(df)

  df.to_csv(os.path.join(_PROCESSED_DATA_DIR, file), index=True)

Let's review the contents of the files again

In [20]:
csv_file_list, csv_full_path_list = UNDP_utils.generate_file_list_from_dir(path=_PROCESSED_DATA_DIR, filter = '.csv', display=False)

In [21]:
for file in csv_full_path_list:
  temp_df = pd.read_csv(file)
  temp_df = temp_df.set_index('Country')
  print(f'DataFrame: {file}')
  temp_df.head(5)
  print ('\n')
  temp_df.describe().transpose()
  print('\n')
  temp_df.info()
  print(100 * '-')
  print('\n\n')

DataFrame: ./processed_data/Urban_Population.csv


,1990,1995,2000,2005,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
Country,,,,,,,,,,,,,,
Afghanistan,21.2,21.6,22.1,22.7,23.7,23.9,24.2,24.4,24.6,24.8,25.0,25.2,25.5,25.8
Albania,36.4,38.9,41.7,46.7,52.2,53.2,54.3,55.4,56.4,57.4,58.4,59.4,60.3,61.2
Algeria,52.1,56.0,59.9,63.8,67.5,68.2,68.9,69.6,70.2,70.8,71.5,72.1,72.6,73.2
Andorra,94.7,93.7,92.4,90.3,88.8,88.7,88.6,88.5,88.4,88.3,88.2,88.2,88.1,88.0
Angola,37.1,44.2,50.1,56.0,59.8,60.5,61.3,62.0,62.7,63.4,64.1,64.8,65.5,66.2


,count,mean,std,min,25%,50%,75%,max
1990,195.0,50.499487,24.034581,5.4,29.70,49.4,69.55,100.0
1995,195.0,51.951795,23.803676,7.2,32.00,51.7,71.75,100.0
2000,195.0,53.278974,23.605607,8.2,33.10,53.3,72.85,100.0
2005,195.0,54.845641,23.456233,9.4,35.25,55.6,73.50,100.0
2010,195.0,56.451795,23.439469,10.6,36.20,56.8,74.90,100.0
2011,195.0,56.767692,23.428246,10.9,36.80,56.9,75.40,100.0
2012,195.0,57.076923,23.416248,11.2,37.45,57.1,76.00,100.0
2013,195.0,57.378974,23.400388,11.5,38.05,57.3,76.75,100.0
2014,195.0,57.681026,23.389301,11.8,38.65,57.5,77.05,100.0
2015,195.0,57.992821,23.361005,12.1,39.35,57.7,77.20,100.0




<class 'pandas.core.frame.DataFrame'>
Index: 195 entries,  Afghanistan to  Zimbabwe
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   1990    195 non-null    float64
 1   1995    195 non-null    float64
 2   2000    195 non-null    float64
 3   2005    195 non-null    float64
 4   2010    195 non-null    float64
 5   2011    195 non-null    float64
 6   2012    195 non-null    float64
 7   2013    195 non-null    float64
 8   2014    195 non-null    float64
 9   2015    195 non-null    float64
 10  2016    195 non-null    float64
 11  2017    195 non-null    float64
 12  2018    195 non-null    float64
 13  2019    195 non-null    float64
dtypes: float64(14)
memory usage: 22.9+ KB
----------------------------------------------------------------------------------------------------



DataFrame: ./processed_data/Total_Population.csv


,1990,1995,2000,2005,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2030
Country,,,,,,,,,,,,,,,
Afghanistan,12.4,18.1,20.8,25.7,29.2,30.1,31.2,32.3,33.4,34.4,35.4,36.3,37.2,38.0,48.1
Albania,3.3,3.1,3.1,3.1,2.9,2.9,2.9,2.9,2.9,2.9,2.9,2.9,2.9,2.9,2.8
Algeria,25.8,28.8,31.0,33.1,36.0,36.7,37.4,38.1,38.9,39.7,40.6,41.4,42.2,43.1,50.4
Andorra,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
Angola,11.8,13.9,16.4,19.4,23.4,24.2,25.1,26.0,26.9,27.9,28.8,29.8,30.8,31.8,44.8


...

,count,mean,std,min,25%,50%,75%,max
1990,185.0,3.465946,13.533677,0.0,0.2,0.7,2.3,133.0
1995,185.0,3.334054,12.134141,0.0,0.2,0.7,2.6,124.8
2000,185.0,3.317838,11.839495,0.0,0.2,0.7,2.7,127.8
2005,185.0,3.380541,11.779729,0.0,0.2,0.7,2.7,129.5
2010,185.0,3.515135,11.819636,0.0,0.2,0.7,2.9,128.2
2011,185.0,3.547568,11.819067,0.0,0.2,0.7,2.9,127.2
2012,185.0,3.574054,11.755345,0.0,0.2,0.7,3.0,125.3
2013,185.0,3.589189,11.651786,0.0,0.2,0.7,3.0,122.9
2014,185.0,3.603784,11.560001,0.0,0.2,0.8,2.9,120.7
2015,185.0,3.617297,11.483633,0.0,0.2,0.8,2.9,119.0




<class 'pandas.core.frame.DataFrame'>
Index: 185 entries,  Afghanistan to  Zimbabwe
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   1990    185 non-null    float64
 1   1995    185 non-null    float64
 2   2000    185 non-null    float64
 3   2005    185 non-null    float64
 4   2010    185 non-null    float64
 5   2011    185 non-null    float64
 6   2012    185 non-null    float64
 7   2013    185 non-null    float64
 8   2014    185 non-null    float64
 9   2015    185 non-null    float64
 10  2016    185 non-null    float64
 11  2017    185 non-null    float64
 12  2018    185 non-null    float64
 13  2019    185 non-null    float64
dtypes: float64(14)
memory usage: 21.7+ KB
----------------------------------------------------------------------------------------------------



DataFrame: ./processed_data/Population _ages_65 _and _older.csv


,1990,1995,2000,2005,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
Country,,,,,,,,,,,,,,
Afghanistan,0.3,0.4,0.5,0.6,0.7,0.7,0.7,0.8,0.8,0.9,0.9,0.9,1.0,1.0
Albania,0.2,0.2,0.2,0.3,0.3,0.3,0.3,0.3,0.4,0.4,0.4,0.4,0.4,0.4
Algeria,0.9,1.0,1.3,1.6,2.0,2.0,2.1,2.1,2.2,2.3,2.4,2.6,2.7,2.8
Angola,0.3,0.4,0.4,0.5,0.6,0.6,0.6,0.6,0.6,0.6,0.7,0.7,0.7,0.7
Antigua and Barbuda,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,count,mean,std,min,25%,50%,75%,max
1990,185.0,1.763243,6.202695,0.0,0.1,0.3,1.0,66.3
1995,185.0,2.013514,7.067930,0.0,0.1,0.4,1.1,76.0
2000,185.0,2.264865,8.069225,0.0,0.1,0.4,1.3,87.9
2005,185.0,2.549189,9.141622,0.0,0.1,0.4,1.4,99.6
2010,185.0,2.821622,10.188301,0.0,0.1,0.5,1.6,110.5
2011,185.0,2.895135,10.484073,0.0,0.1,0.5,1.6,113.7
2012,185.0,2.976216,10.787653,0.0,0.1,0.5,1.7,117.0
2013,185.0,3.061622,11.124107,0.0,0.1,0.5,1.8,120.8
2014,185.0,3.155135,11.525265,0.0,0.1,0.5,1.8,125.5
2015,185.0,3.257297,12.001008,0.0,0.1,0.5,1.9,131.3




<class 'pandas.core.frame.DataFrame'>
Index: 185 entries,  Afghanistan to  Zimbabwe
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   1990    185 non-null    float64
 1   1995    185 non-null    float64
 2   2000    185 non-null    float64
 3   2005    185 non-null    float64
 4   2010    185 non-null    float64
 5   2011    185 non-null    float64
 6   2012    185 non-null    float64
 7   2013    185 non-null    float64
 8   2014    185 non-null    float64
 9   2015    185 non-null    float64
 10  2016    185 non-null    float64
 11  2017    185 non-null    float64
 12  2018    185 non-null    float64
 13  2019    185 non-null    float64
dtypes: float64(14)
memory usage: 21.7+ KB
----------------------------------------------------------------------------------------------------





In [22]:
# 3. Interpolate values for median_age dataframe and save to file
df = pd.read_csv(os.path.join(_PROCESSED_DATA_DIR, 'Median_age.csv'))
df = df.set_index('Country')

new_cols = ['1990', '1995', '2000', '2005', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019']
df = interpolate_columns(df, new_cols)
df.to_csv(os.path.join(_PROCESSED_DATA_DIR, 'Median_age.csv'), index=True)

In [23]:
#4 - Drop the 2030 column from total population as store it as a label dataframe
df = pd.read_csv('./raw_data/Total_Population.csv')
df = df.set_index('Country')

y_pop_2030 = df['2030']
df = df.drop('2030', axis=1)

df.to_csv('./raw_data/Total_Population.csv', index=True)
y_pop_2030.to_csv('./processed_data/y_pop_2030.csv', index=True)

In [24]:
for file in csv_full_path_list:
  temp_df = pd.read_csv(file)
  print (f"csv file: {file} - columns: {temp_df.columns}")

csv file: ./processed_data/Urban_Population.csv - columns: Index(['Country', '1990', '1995', '2000', '2005', '2010', '2011', '2012',
       '2013', '2014', '2015', '2016', '2017', '2018', '2019'],
      dtype='object')
csv file: ./processed_data/Total_Population.csv - columns: Index(['Country', '1990', '1995', '2000', '2005', '2010', '2011', '2012',
       '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2030'],
      dtype='object')
csv file: ./processed_data/Young_age_dependency_ratio.csv - columns: Index(['Country', '1990', '1995', '2000', '2005', '2010', '2011', '2012',
       '2013', '2014', '2015', '2016', '2017', '2018', '2019'],
      dtype='object')
csv file: ./processed_data/Median_age.csv - columns: Index(['Country', '1990', '1995', '2000', '2005', '2010', '2015', '2020',
       '2011', '2012', '2013', '2014', '2016', '2017', '2018', '2019'],
      dtype='object')
csv file: ./processed_data/Population_ages_15_64.csv - columns: Index(['Country', '1990', '1995', '2000'

**Next Task**
1.   Merge all the dataframes into one and save into the processed_data directory



In [25]:
# 1. Merge all the dataframes into one and save into the processed_data directory
res_df = None
for file in csv_file_list:
  df = pd.read_csv(os.path.join(_PROCESSED_DATA_DIR, file))
  df = df.set_index('Country')
  cols = df.columns
  new_cols = [col+"-"+os.path.split(file)[1].replace('.csv',"") for col in cols]
  df.columns = new_cols

  if type(res_df) == type(None):
    res_df = df
  else:
    res_df = pd.merge(res_df, df, how='outer', on='Country')

# Dropping partial rows
res_df = res_df.dropna(axis=0)

res_df.to_csv('./processed_data/UNDP_Demographics_Data.csv', index=True)

In [26]:
shutil.make_archive('./UNDP-Processed', 'zip', _PROCESSED_DATA_DIR)

'/content/UNDP-Processed.zip'